# Inserting Topic Modeling Output to Clean DataFrame
This Includes Manupulation of columns, mapping and concating of dataframes

### Import Libraries and  reading the necessary data  to dataframes

In [1]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', 1000)

In [2]:
topic_data = pd.read_csv('doc-topics.csv')
clean_data = pd.read_csv('data/clean_data.csv').iloc[:,1:]
topic_data.head()

,docname,topic,proportion
0,review_body.csv:11,2,1.0
1,review_body.csv:46,0,1.0
2,review_body.csv:81,1,1.0
3,review_body.csv:116,0,1.0
4,review_body.csv:151,1,1.0


### Data Manipulating and Cleaning

**Step 1 - Get the Line number for each review from the docname columns, sort and remove row 0 as that is the topic for the  column names**

In [3]:
topic_data['index'] = topic_data['docname'].str.split(':')
topic_data['index'] = topic_data['index'].apply(lambda x: x[1])
topic_data['index']=topic_data['index'].astype(int)
topic_data.sort_values(by=['index'],inplace=True)
topic_data = topic_data.iloc[1:,:]
topic_data.head()

,docname,topic,proportion,index
447,review_body.csv:1,0,1.0,1
469,review_body.csv:2,0,1.0,2
80,review_body.csv:3,0,1.0,3
426,review_body.csv:4,0,1.0,4
348,review_body.csv:5,0,1.0,5


**Step 2 - Seperate rows those proportion is exactly 1 and less then 1. From the rows that are less than one, sort to remove duplicate line numbers. ( There are reviews that have more than one topic present in them. We are onle interested in the dominate topic). Finally combine the dataframe with less than 1 proportion with the dataframe with exactly one proportion**

In [4]:
less_than_1 = topic_data[topic_data['proportion']<1]
equal_to_1 = topic_data[topic_data['proportion']==1]
less_than_1.sort_values('proportion',ascending=False,inplace=True)
less_than_1.reset_index(inplace=True,drop=True)

/Users/anitaokoh/anaconda3/envs/temporary_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
less_than_1 = less_than_1.iloc[:47,:]
topics = pd.concat([less_than_1,equal_to_1],ignore_index=True)

**Step 3 - Resort and reindex the new dataframe created. Then finally concat the dataframe with the original clean data that contains the reviews**

In [6]:
topics.sort_values('index',inplace=True)
topics['index'] = range(0,653)
topics.reset_index(inplace=True,drop=True)
topics.head()

,docname,topic,proportion,index
0,review_body.csv:1,0,1.0,0
1,review_body.csv:2,0,1.0,1
2,review_body.csv:3,0,1.0,2
3,review_body.csv:4,0,1.0,3
4,review_body.csv:5,0,1.0,4


In [7]:
result = pd.concat([clean_data, topics], axis=1, sort=False)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 653 entries, 0 to 652
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   review_title    653 non-null    object 
 1   review_body     653 non-null    object 
 2   company_source  653 non-null    object 
 3   docname         653 non-null    object 
 4   topic           653 non-null    int64  
 5   proportion      653 non-null    float64
 6   index           653 non-null    int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 35.8+ KB


**Step 4 - Rename the topics into categories and map it into the dataframe. Indicate reviews that have more than one topics. Drop unneccessary columns**

In [9]:
mapping_topic = {0:'About Product/Services',1:'Experience/Expectation',2:'Transaction Journey'}
result['topic'] = result['topic'].map(mapping_topic)
result['topic'] = np.where(result['proportion']==1,result['topic'],result['topic']+'+')
result.head()

,review_title,review_body,company_source,docname,topic,proportion,index
0,I ordered an item and had a little,I ordered an item and had a little payment mis...,zalando,review_body.csv:1,About Product/Services,1.0,0
1,I ordered few items for 800 euro,I ordered few items for 800 euro came only che...,zalando,review_body.csv:2,About Product/Services,1.0,1
2,Always the first choice for apparel shopping,Always the first choice for apparel shopping i...,zalando,review_body.csv:3,About Product/Services,1.0,2
3,Support is bad return items and do not get mon...,Ordered items for 1200 got them all but in the...,zalando,review_body.csv:4,About Product/Services,1.0,3
4,Utterly useless in every respect,2 items ordered the first came the next day dh...,zalando,review_body.csv:5,About Product/Services,1.0,4


In [10]:
result.drop(['docname','proportion'],axis=1,inplace=True)

In [11]:
result.to_csv('topicmodelling_data.csv')

The final dataframe is then stored in a csv for further merging of the sentiment results